In [2]:
import numpy as np 
import pandas as pd 
import glob
from tqdm.notebook import tqdm
import os

In [3]:
def std(x):
    return (x - np.mean(x)) / np.std(x)

In [4]:
SUBMISSIONS = {
    # CITE 任务集成，权重和为1
    '../result/cite_sub/LGBM_submission.csv':0.2,
    '../result/cite_sub/GRU_submission.csv':0.8,
    
    # multi 任务集成，权重和为1
    '../result/multi_sub/MLP_submission.csv':1,
}

In [5]:
cell_ids = pd.read_parquet('../data/multimodal-single-cell-as-sparse-matrix/evaluation.parquet').cell_id

In [6]:
def gen_std_submission(path, cell_ids):
    """
    Standardize submission per cell_id
    """
    df = pd.read_csv(path)
    df['cell_id'] = cell_ids    
    vals = []
    for idx, g in tqdm(df.groupby('cell_id', sort=False), desc=f'Standardizing {path}', miniters=1000):
        vals.append(std(g.target).values)
    vals = np.concatenate(vals)
    return vals

In [7]:
def gen_ensemble(technology):
    for path in tqdm([path for path in SUBMISSIONS.keys() if technology in path], desc='Process submission'):
        weight = SUBMISSIONS[path]
        ensemble = gen_std_submission(path, cell_ids) * weight

    return ensemble

In [8]:
PRED_SEGMENTS = [(0, 6812820), (6812820, 65744180)]
ensemble = []
for tech, (from_idx, to_idx) in tqdm(list(zip(['cite', 'multi'], PRED_SEGMENTS)), desc='Technology'):    
    ensemble.append(gen_ensemble(tech)[from_idx: to_idx])
    
    
ensemble = np.concatenate(ensemble)

Technology:   0%|          | 0/2 [00:00<?, ?it/s]

Process submission:   0%|          | 0/2 [00:00<?, ?it/s]

Standardizing ../result/cite_sub/LGBM_submission.csv:   0%|          | 0/65443 [00:00<?, ?it/s]

Standardizing ../result/cite_sub/GRU_submission.csv:   0%|          | 0/65443 [00:00<?, ?it/s]

Process submission:   0%|          | 0/1 [00:00<?, ?it/s]

Standardizing ../result/multi_sub/MLP_submission.csv:   0%|          | 0/65443 [00:00<?, ?it/s]

In [9]:
df_submit = pd.read_parquet('../data/multimodal-single-cell-as-sparse-matrix/sample_submission.parquet')
df_submit['target'] = ensemble
df_submit.to_csv('../result/sub/submission.csv', index=False)
df_submit

,row_id,target
0,0,-0.441313
1,1,-0.409348
2,2,-0.286577
3,3,1.066649
4,4,1.089869
...,...,...
65744175,65744175,NaN
65744176,65744176,NaN
65744177,65744177,NaN
65744178,65744178,NaN


In [12]:
pd.read_csv("../result/multi_sub/MLP_submission.csv")

,target
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
65744175,NaN
65744176,NaN
65744177,NaN
65744178,NaN
